In [134]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [99]:
df = pd.read_csv(r'C:\Users\Selva\Desktop\Cyrus\OneDrive_1_9-4-2019\Predict rating.csv')

In [100]:
pd.set_option('display.max_colwidth', -1)

In [101]:
df[['review_title','review_body','rating','staff_rating','atmos_rating','bud_rating']].sample(25)

,review_title,review_body,rating,staff_rating,atmos_rating,bud_rating
86672,caviar pre roll,Best pre roll I've had since I've been here,5.0,5.0,5.0,5.0
62367,TASTIEST & MOST POTENT,"Been using C.O.D. services for a while, and have never been disappointed.\nAwesome selection, great tasty/fruity exotic strains, and the potency is definitely top-shelf.\nGreat delivery as well, accurate ETA and regular updates on the way.\nTruly the best all around, no doubt !",5.0,5.0,NaN,5.0
83075,Best Edible Pricing,"If edibles are your thing, heck even if they aren't, this place has the best pricing and a huge selection of options! Make sure to see Jay and tell I sent you!",5.0,5.0,5.0,5.0
54931,Coco Diaz Original Spot,"I just moved to the area and I was on the hunt for a top shelf collective to get my meds. Parking for this place is a little rough because it's in a relatively high traffic part of NoHo, but there's metered parking along the street. Inside the place is well decorated and air conditioned. The staff was friendly and helpful, they made an effort to make sure that I felt like I had a good experience. Great customer service and the recommendations seemed like they weren't trying to bullshit me and tell me 'everything we sell is potent.' They have a big selection of buds, edibles and a moderate selection of concentrates. Prices weren't cheap but they weren't a rip off either; if you're trying to get an ounce for 100 dollars don't bother coming here. I think the cost justified the price of the stuff they had. Wil be returning regularly.",4.6,5.0,5.0,5.0
138493,Review,Greatest bud tenders good selection of flower and welcoming environment,5.0,5.0,5.0,5.0
139301,Thank you Cindy!!,Such a personable bud tender and always answers questions. Great product!! Thanks guys!,5.0,5.0,5.0,5.0
93776,hydro buds and that evolution SHATTER. wowwww,this is one of the best im thinking maybe even the best delivery in the area. the guys who deliver are all super nice and i never wait too long so thats a big plus. and the prices i think are fair for how potent the meds are. so getting to the point i was blown away by this service last time i ordered and had to leave a review when someone really is on top of the game like sac confidential. i like good buds and these guys have them talking those nice dense hyrdo buds but more and more im turning into a wax and shatter dabber as they call em not even smoking weed as much. i love the strong concentrates and especially shatter because thats the best tasting and pure. that evolution shatter you all had blew me away for real i never tried anything like that before and i go out to oakland sometimes for meds so that says something. very happy patient here not one bad thing to say about this service the others guys could learn something haha,5.0,5.0,NaN,5.0
92127,bomb shop in I. E,in El Monte and I can't wait to go to wonderland they have ð¥ð¥ð¥ð¥ð¥ð¥,5.0,5.0,5.0,5.0
97141,Releaf,Best prices great and friendly staff. I'm omw in now. Visit CC Releaf Today!! ð¯ð,5.0,5.0,5.0,5.0
172195,good weed,girls are supper friendly and act professional,5.0,5.0,5.0,5.0


## 1.Predict Rating

In [117]:
rating_df= df[['review_title','review_body','rating']]

In [118]:
rating_df.head()

,review_title,review_body,rating
0,BEST QUALITY FOR BEAUTIFUL PRICE,Best quality of bud i have ever seen every jar was filled with sticky potent weed not to mention the fire waxes! My new spot fosho,5.0
1,FTP,"Cool shop. Nice meds, prices, and staff.",5.0
2,great place,good product great staff friendly atmosphere,5.0
3,OC HOTBOX GOT ð¥,"Bomb meds and staff, headed to the hot box now and so should you!",5.0
4,Great place.,"1st time client. Easy location, great atmosphere. The staff was pleasant and I worked with Tyler who was very knowledgable about his products. I felt comfortable purchasing, definitely will return.",5.0


#### Checking null values

In [119]:
rating_df.isnull().values.any()

True

In [120]:
rating_df.shape

(173101, 3)

In [121]:
rating_df = rating_df.dropna()
rating_df.reset_index(inplace=True,drop=True)

In [122]:
rating_df.shape

(161131, 3)

In [123]:
rating_df.isnull().values.any()

False

#### Cleaning Data

In [124]:
for index,row in rating_df.iterrows():
    rating_df.at[index,'review_title'] = ''.join([char if ord(char) < 128 else '' for char in row['review_title']])
    rating_df.at[index,'review_body'] = ''.join([char if ord(char) < 128 else '' for char in row['review_body']])

In [125]:
rating_df['review_title'].replace('',np.nan,inplace=True)
rating_df['review_body'].replace('',np.nan,inplace=True)

In [126]:
rating_df.isnull().values.any()

True

In [127]:
rating_df.dropna(inplace=True)
rating_df.reset_index(inplace=True,drop=True)

In [128]:
rating_df.shape

(157684, 3)

In [129]:
rating_df.head()

,review_title,review_body,rating
0,BEST QUALITY FOR BEAUTIFUL PRICE,Best quality of bud i have ever seen every jar was filled with sticky potent weed not to mention the fire waxes! My new spot fosho,5.0
1,FTP,"Cool shop. Nice meds, prices, and staff.",5.0
2,great place,good product great staff friendly atmosphere,5.0
3,OC HOTBOX GOT,"Bomb meds and staff, headed to the hot box now and so should you!",5.0
4,Great place.,"1st time client. Easy location, great atmosphere. The staff was pleasant and I worked with Tyler who was very knowledgable about his products. I felt comfortable purchasing, definitely will return.",5.0


#### Generating Features

In [130]:
rating_df['review'] =  rating_df['review_title'] + ' ' + rating_df['review_body']
rating_df = rating_df[['review','rating']]

In [131]:
rating_df.head()

,review,rating
0,BEST QUALITY FOR BEAUTIFUL PRICE Best quality of bud i have ever seen every jar was filled with sticky potent weed not to mention the fire waxes! My new spot fosho,5.0
1,"FTP Cool shop. Nice meds, prices, and staff.",5.0
2,great place good product great staff friendly atmosphere,5.0
3,"OC HOTBOX GOT Bomb meds and staff, headed to the hot box now and so should you!",5.0
4,"Great place. 1st time client. Easy location, great atmosphere. The staff was pleasant and I worked with Tyler who was very knowledgable about his products. I felt comfortable purchasing, definitely will return.",5.0


In [144]:
vect = TfidfVectorizer(input='content',stop_words='english',analyzer='word', ngram_range=(1,6),
                     min_df = 0, sublinear_tf=True)
X = vect.fit_transform(rating_df.review)


C:\Users\Selva\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [145]:
print(X.shape)

(157684, 7738982)


#### Modelling